In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## 1. 서울 + 결측치


## 2. 열제거 (PM10, PM2.5 제외 제거하기)

## 3. average of 11am , 12pm

## 4. 지오코딩

In [2]:
pip install pandas

한 연도 안에서 지역 분류, 결측치 제거 + PM 값은 10시와 11시 값의 평균값 구해서 넣기

In [ ]:
import os
import pandas as pd

# 5개년 데이터를 저장한 폴더 경로
project_folder = "./에어코리아"
address_file = "./기상관측데이터/기상관측데이터_지점정리.csv"
output_folder = "./에어코리아"
address = pd.read_csv(address_file)
location_names = address['지점명'].dropna().unique()

# 결과 저장용 데이터프레임 초기화
all_results = pd.DataFrame()

# 연도별 데이터 처리
for year_folder in ['2019', '2020', '2021', '2022', '2023']:
    year_folder_path = os.path.join(project_folder, year_folder)

    if os.path.exists(year_folder_path):
        print(f"Processing folder: {year_folder_path}")

        for filename in os.listdir(year_folder_path):
            if filename.endswith(".xlsx"):
                file_path = os.path.join(year_folder_path, filename)
                df = pd.read_excel(file_path, engine='openpyxl')

                for location_name in location_names:
                    # 특정 지점명으로 데이터 필터링
                    df_filtered = df[df['주소'].str.contains(location_name, na=False)]
                    if df_filtered.empty:
                        continue

                    # 새로운 열에 지점명 추가
                    # 수정된 코드
                    df_filtered.loc[:, '지점명'] = location_name
                    
                    # 결측치 제거 (PM10 또는 PM2.5 값이 NaN인 행 제거)
                    df_filtered = df_filtered.dropna(subset=['PM10', 'PM25'])

                    # '측정일시'를 문자열로 변환 후 시간 필터링
                    df_filtered['측정일시'] = df_filtered['측정일시'].astype(str)
                    df_filtered['시간'] = df_filtered['측정일시'].str[-2:].astype(int)

                    # 10시와 11시 데이터만 선택
                    df_filtered = df_filtered[df_filtered['시간'].isin([10, 11])]
                    
                    # 측정일시에서 날짜만 추출
                    df_filtered['측정일시_날짜'] = df_filtered['측정일시'].str[:8]

                    # 불필요한 열 제거
                    drop_columns = ['지역', '망', '측정소명', 'SO2', 'CO', 'O3', 'NO2']
                    df_filtered = df_filtered.drop(columns=[col for col in drop_columns if col in df_filtered.columns])

                    # 평균 계산
                    grouped_df = df_filtered.groupby(
                        ['지점명', '측정소코드', '측정일시_날짜', '주소']
                    )[['PM10', 'PM25']].mean().reset_index()

                    # 결과 데이터프레임에 추가
                    all_results = pd.concat([all_results, grouped_df], ignore_index=True)

# 동일 연도 데이터끼리 파일로 저장
all_results['측정일시_년도'] = all_results['측정일시_날짜'].str[:4]

for year, year_data in all_results.groupby('측정일시_년도'):
    output_file = os.path.join(output_folder, f"{year}_data.csv")
    year_data.to_csv(output_file, index=False, encoding='utf-8-sig')

print("작업이 완료되었습니다!")


In [4]:
if all_results.isna().any().any():
    print("데이터프레임에 결측치가 존재합니다.")
else:
    print("데이터프레임에 결측치가 없습니다.")

데이터프레임에 결측치가 없습니다.


각 지역 별로 나누기

In [8]:
import pandas as pd
import os

# 데이터 폴더 경로
input_folder = "./에어코리아"
output_folder = "./에어코리아"

# 기상관측소 파일 로드
station_file = "./기상관측데이터/기상관측데이터_지점정리.csv"
station_data = pd.read_csv(station_file, encoding='utf-8-sig')

# '지점명'과 '주소' 매핑 테이블 생성
station_names = station_data['지점명'].dropna().unique()  # 유니크한 지점명만 활용

# 연도별로 데이터 분리 및 처리
for year_file in os.listdir(input_folder):
    if year_file.endswith(".csv"):
        # 연도 가져오기 (예: 2019_data.csv -> 2019)
        year = year_file.split("_")[0]
        
        # 연도별 데이터 읽기
        file_path = os.path.join(input_folder, year_file)
        year_data = pd.read_csv(file_path, encoding='utf-8-sig')
        
        # 각 지점명별로 데이터 분리
        for station_name in station_names:
            # 주소에 지점명이 포함된 데이터 필터링
            group = year_data[year_data['주소'].str.contains(station_name, na=False)]
            
            if group.empty:
                continue  # 해당 지점명이 포함된 데이터가 없으면 건너뛰기
            
            # 출력 폴더 생성 (지점명별 폴더)
            location_folder = os.path.join(output_folder, station_name)
            os.makedirs(location_folder, exist_ok=True)
            
            # 파일 이름: 연도.csv (예: 2019.csv)
            output_file = os.path.join(location_folder, f"{year}.csv")
            
            # 데이터 저장
            group.to_csv(output_file, index=False, encoding='utf-8-sig')

print("지점명별 연도별 CSV 파일 생성이 완료되었습니다!")


지점명별 연도별 CSV 파일 생성이 완료되었습니다!


바로 아래 코드는 csv 파일 정리를 하느라, 특정 열 없애는 데에 쓴 거. 다른 때에는 필요 없음.

In [ ]:

# 지역 폴더들이 모여 있는 상위 폴더 경로
base_folder = "./에어코리아/지역별로_폴더분류"

# 모든 하위 폴더 탐색
for root, dirs, files in os.walk(base_folder):
    for file in files:
        if file.endswith(".csv"):  # CSV 파일만 처리
            file_path = os.path.join(root, file)
            
            # CSV 파일 읽기
            df = pd.read_csv(file_path, encoding='utf-8-sig')
            
            # '지점명' 열이 존재하면 삭제
            if '측정일시_년도' in df.columns:
                df = df.drop(columns=['측정일시_년도'])
                print(f"'측정일시_년도' 열 삭제: {file_path}")
                
                # 수정된 데이터 다시 저장
                df.to_csv(file_path, index=False, encoding='utf-8-sig')

print("모든 CSV 파일에서 '측정일시_년도' 열 삭제 완료!")


이 아래는 안 건들임

## 5. 센티넬이랑 matching

In [ ]:
# 미리 처리된 데이터프레임 로드
df = pd.read_csv('/content/drive/MyDrive/대기오염 Data/New_airkorea/preprocessing/2023_pre.csv')

# 새로운 specific_dates 리스트
specific_dates = [
    '20230815', '20230909', '20230924', '20231029', '20231113',
    '20231118', '20231123', '20231128', '20231203', '20231208',
    '20231223', '20231228',
]

# 새로운 파일 경로 리스트
filtered_file_paths = [
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20230815.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20230909.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20230924.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231029.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231113.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231118.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231123.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231128.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231203.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231208.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231223.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231228.csv',
]

# Loop through each specific date and corresponding file path
for i, specific_date in enumerate(specific_dates):
    # Filter the dataframe by the specified date (YYYYMMDD)
    filtered_df = df[df['측정일시_날짜'].astype(str).str[:8].isin([specific_date])]

    # Save the filtered dataframe to a new CSV file
    filtered_df.to_csv(filtered_file_paths[i], index=False)

    # Print confirmation
    print(f"Filtered data saved to {filtered_file_paths[i]}")


Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20230815.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20230909.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20230924.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231029.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231113.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231118.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231123.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231128.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231203.csv
Filtered data saved to /content/drive/MyDrive/대기오염

## 6. 센티넬이랑 합치기

In [ ]:
import pandas as pd
import os

# 파일 경로 설정
airkorea_folder = '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching'
sentinel_folder = '/content/drive/MyDrive/대기오염 Data/Sentinel 2 Data/band value/2023/'
output_folder = '/content/drive/MyDrive/대기오염 Data/Sentinel+Airkorea'

# 파일 목록 가져오기
airkorea_files = [f for f in os.listdir(airkorea_folder) if f.endswith('.csv')]
sentinel_files = [f for f in os.listdir(sentinel_folder) if f.endswith('.csv')]

# 공통 파일 목록 찾기
common_files = set(airkorea_files) & set(sentinel_files)

# 병합 기준 컬럼 설정
merge_columns = ['Latitude', 'Longitude']

# 파일 처리
for file_name in common_files:
    try:
        # 파일 읽기
        df_airkorea = pd.read_csv(os.path.join(airkorea_folder, file_name), index_col=0)
        df_sentinel = pd.read_csv(os.path.join(sentinel_folder, file_name))

        # 데이터 병합
        df_merged = pd.merge(df_airkorea, df_sentinel[merge_columns + ['geometry', 'AOT','Band1', 'Band2', 'Band3', 'Band4', 'Band5', 'Band6', 'Band7', 'Band11', 'Band12', 'B8A(NIR)', 'SCL', 'TCI', 'WVP']],
                             on=merge_columns, how='left')
        # 데이터 병합
        #df_merged = pd.merge(df_airkorea, df_sentinel[merge_columns + ['geometry', 'AOT','Band2', 'Band3', 'Band4', 'Band5', 'Band6', 'Band7', 'Band11', 'Band12', 'B8A(NIR)', 'SCL', 'TCI', 'WVP']],
        #                     on=merge_columns, how='left')

        # 병합된 데이터프레임을 저장
        df_merged.to_csv(os.path.join(output_folder, file_name), index=False)

        # 병합 결과 출력 (첫 몇 줄)
        print(f"Processed and saved {file_name}")

    except KeyError as e:
        # 오류 발생 시 메시지 출력 후 넘어가기
        print(f"KeyError: {e} in file {file_name}. Skipping this file.")

    except Exception as e:
        # 기타 오류 발생 시 메시지 출력 후 넘어가기
        print(f"Error: {e} in file {file_name}. Skipping this file.")

print("All files processed.")


Processed and saved 20230706.csv
Processed and saved 20231113.csv
Processed and saved 20230427.csv
Processed and saved 20230201.csv
Processed and saved 20231223.csv
Processed and saved 20230303.csv
Processed and saved 20230402.csv
Processed and saved 20231228.csv
Processed and saved 20231208.csv
Processed and saved 20230112.csv
Processed and saved 20230417.csv
Processed and saved 20231118.csv
Processed and saved 20231123.csv
Processed and saved 20230517.csv
Processed and saved 20230701.csv
Processed and saved 20230815.csv
Processed and saved 20230522.csv
Processed and saved 20230412.csv
Processed and saved 20230721.csv
Processed and saved 20230313.csv
Processed and saved 20230226.csv
Processed and saved 20230127.csv
Processed and saved 20230616.csv
Processed and saved 20230407.csv
Processed and saved 20231029.csv
Processed and saved 20230909.csv
Processed and saved 20231128.csv
Processed and saved 20230221.csv
Processed and saved 20230924.csv
Processed and saved 20230102.csv
Processed 